# Práctica 3: Procesamiento de vídeos

Librerías:

In [1]:
import numpy as np
import cv2

### Ejercicio 1: Implementación de un detector de movimiento

Para el video 1

In [11]:
cap = cv2.VideoCapture('video1.mkv')
ret, pframe = cap.read()
(h, w, c) = pframe.shape
thr = 20
writer = cv2.VideoWriter('video1_movement.mkv', cv2.VideoWriter_fourcc(*"XVID"),30, (w, h))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        res = cv2.absdiff(frame, pframe)  #Calculamos la diferencia con respecto al frame anterior
        res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY) #Pasamos a escala de grises
        
        aux = np.zeros((h,w))
        for i in range(h):
            for j in range(w):  #Por cada posición calculamos el promedio de su vecindad (3x3), incluido los bordes.
                aux[i,j] = np.mean([res[x,y] for x in range(i-1,i+2) for y in range(j-1, j+2) if (x >= 0 and x < h and y >= 0 and y < w)])
                
        
        res = np.array(aux > thr, dtype='uint8')*255  #Umbralizamos
        res = np.broadcast_to(res[:,:,np.newaxis], (h, w, c)) #Broadcast a 3 canales
        writer.write(res)
        pframe = frame
    else:
        break

cap.release()
writer.release()

Para el video 2

In [3]:
cap = cv2.VideoCapture('video2.mkv')
ret, pframe = cap.read()
(h, w, c) = pframe.shape
thr = 20
writer = cv2.VideoWriter('video2_movement.mkv', cv2.VideoWriter_fourcc(*"XVID"),30, (w, h))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        res = cv2.absdiff(frame, pframe) #Calculamos la diferencia con respecto al frame anterior
        res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY) #Pasamos a escala de grises
        
        aux = np.zeros((h,w))
        for i in range(h):
            for j in range(w): #Por cada posición calculamos el promedio de su vecindad (3x3), incluido los bordes.
                aux[i,j] = np.mean([res[x,y] for x in range(i-1,i+2) for y in range(j-1, j+2) if (x >= 0 and x < h and y >= 0 and y < w)])
                
        
        res = np.array(aux > thr, dtype='uint8')*255 #Umbralizamos
        res = np.broadcast_to(res[:,:,np.newaxis], (h, w, c)) #Broadcast a 3 canales
        writer.write(res)
        pframe = frame
    else:
        break

cap.release()
writer.release()

### Ejercicio 2: Implementación de un segmentador de escenas

Para el video 1

In [16]:
cap = cv2.VideoCapture('video1.mkv')
ret, frame = cap.read()
(h, w, c) = frame.shape
font = cv2.FONT_HERSHEY_TRIPLEX
white = (255,255,255)
(scene, cont) = (1, 1) #Inicializamos el contador de escenas y de frames en buffer a 1
(thr, buffsize) = (30, 30) #Especificamos el umbral y tamaño del buffer
buffer = np.zeros((buffsize,h,w,c)) #Creación del buffer
buffer[0] = frame   #Insertamos la primera frame

writer = cv2.VideoWriter('video1_scene.mkv', cv2.VideoWriter_fourcc(*"XVID"),30, (w, h))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        res = frame.copy()
        if cont == buffsize:  #El buffer está lleno
            
            frame_ref = np.zeros((h,w,c), dtype='float32')
            for item in buffer[:buffsize//2]:  #Tomamos la mitad inicial de frames y sacamos su frame promedio
                frame_ref += item
            frame_ref /= (buffsize/2)
            
            frame_act = np.zeros((h,w,c), dtype='float32')
            for item in buffer[buffsize//2:]:  #Tomamos la mitad final de frames y sacamos su frame promedio
                frame_act += item
            frame_act /= (buffsize/2)
            
            img_diff = np.sum(cv2.absdiff(frame_ref, frame_act))/(w*h*3)   #Calculamos la diferencia media
            
            if (img_diff > thr): #Si supera el umbral, añadir 1 al condador de escenas y vaciar el buffer
                scene = scene + 1
                buffer = np.zeros((buffsize,h,w,c))
                buffer[0] = frame  #Metemos la frame actual en el buffer vacio
                cont = 1  #Inicializamos el contador de frames a 1
            
            else:  #Si no supera el umbral, quitamos el frame más antiguo en el buffer (primera posición) y añadimos el frame actual
                buffer = np.delete(buffer, 0, 0)
                buffer = np.append(buffer,res[np.newaxis,:,:,:],axis = 0)
                
        
        else: #Si el buffer no está lleno, añadimos frames hasta que lo esté
            buffer[cont] = frame
            cont += 1
    
        
        text = "Scene " + str(scene)  #Incluimos como texto el número de escena en el frame actual 
        cv2.putText(res, text, (150, 250), font, 1.5, white)
        writer.write(res)

    else:
        break

cap.release()
writer.release()


Para el video 3

In [15]:
cap = cv2.VideoCapture('video3.mkv')
ret, frame = cap.read()
(h, w, c) = frame.shape
font = cv2.FONT_HERSHEY_TRIPLEX
white = (255,255,255)
(scene, cont) = (1, 1) #Inicializamos el contador de escenas y de frames en buffer a 1
(thr, buffsize) = (30, 30)  #Especificamos el umbral y tamaño del buffer
buffer = np.zeros((buffsize,h,w,c)) #Creación del buffer
buffer[0] = frame  #Insertamos la primera frame

writer = cv2.VideoWriter('video3_scene.mkv', cv2.VideoWriter_fourcc(*"XVID"),30, (w, h))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        res = frame.copy()
        if cont == buffsize: #El buffer está lleno
            
            frame_ref = np.zeros((h,w,c), dtype='float32')
            for item in buffer[:buffsize//2]:  #Tomamos la mitad inicial de frames y sacamos su frame promedio
                frame_ref += item
            frame_ref /= (buffsize/2)
            
            frame_act = np.zeros((h,w,c), dtype='float32')  #Tomamos la mitad final de frames y sacamos su frame promedio
            for item in buffer[buffsize//2:]:
                frame_act += item
            frame_act /= (buffsize/2)
            
            img_diff = np.sum(cv2.absdiff(frame_ref, frame_act))/(w*h*3)  #Calculamos la diferencia media
            
            if (img_diff > thr):  #Si supera el umbral, añadir 1 al condador de escenas y vaciar el buffer
                scene = scene + 1
                buffer = np.zeros((buffsize,h,w,c))
                buffer[0] = frame  #Metemos la frame actual en el buffer vacio
                cont = 1  #Inicializamos el contador de frames a 1
            
            else:  #Si no supera el umbral, quitamos el frame más antiguo en el buffer (primera posición) y añadimos el frame actual
                buffer = np.delete(buffer, 0, 0)
                buffer = np.append(buffer,res[np.newaxis,:,:,:],axis = 0)
                
        
        else:  #Si el buffer no está lleno, añadimos frames hasta que lo esté
            buffer[cont] = frame
            cont += 1
    
        
        text = "Scene " + str(scene)  #Incluimos como texto el número de escena en el frame actual 
        cv2.putText(res, text, (150, 250), font, 1.5, white)
        writer.write(res)

    else:
        break

cap.release()
writer.release()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
3

### Ejercicio 3: Implementación de un segmentador foreground / background

Para el video 2

In [2]:
cap = cv2.VideoCapture('video2.mkv')
ret, background_frame = cap.read() # Leemos el primer frame que será considerado el fondo
(h, w, c) = background_frame.shape
(thr,neig) = (80,1)    #Especificamos el umbral y vecindad
writer = cv2.VideoWriter('video2_bgd_for.mkv', cv2.VideoWriter_fourcc(*"XVID"),30, (w, h))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        res = cv2.absdiff(frame, background_frame) #Sacamos la diferencia entre el frame actual y el fondo
        res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY) #Pasamos a escala de grises
        res = np.array(res > thr, dtype='uint8') #Umbralizamos
        
        copy_res = res.copy() #Nos guardamos una copia del resultado
        
        #Filtramos el ruido 
        for i in range(neig,h-neig+1): 
            for j in range(neig,w-neig+1):
                suma = np.sum(res[i-neig:i+neig+1, j-neig:j+neig+1]) #Sumamos por vecindad en la umbralización original
                if suma <= 1: #Si es menor o igual que 1, fijamos a background en la copia
                    copy_res[i,j] = 0
        
        
        broad_res = np.broadcast_to(copy_res[:,:,np.newaxis], (h, w, c)) #Hacemos broadcast del resultado a 3 canales 
        res_final = broad_res * frame  #Multiplicamos por el frame original
        writer.write(res_final)
    
    else:
        break
    
cap.release()
writer.release()

Para el video 4

In [3]:
cap = cv2.VideoCapture('video4.mkv')
ret, background_frame = cap.read() # Leemos el primer frame que será considerado el fondo
(h, w, c) = background_frame.shape
(thr,neig) = (80,1)    #Especificamos el umbral y vecindad
writer = cv2.VideoWriter('video4_bgd_for.mkv', cv2.VideoWriter_fourcc(*"XVID"),30, (w, h))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        res = cv2.absdiff(frame, background_frame)  #Sacamos la diferencia entre el frame actual y el fondo
        res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY) #Pasamos a escala de grises
        res = np.array(res > thr, dtype='uint8') #Umbralizamos
        
        copy_res = res.copy()  #Nos guardamos una copia del resultado
        
        #Filtramos el ruido 
        for i in range(neig,h-neig+1):
            for j in range(neig,w-neig+1):
                suma = np.sum(res[i-neig:i+neig+1, j-neig:j+neig+1])  #Sumamos por vecindad en la umbralización original
                if suma <= 1:  #Si es menor o igual que 1, fijamos a background en la copia
                    copy_res[i,j] = 0
        
        
        broad_res = np.broadcast_to(copy_res[:,:,np.newaxis], (h, w, c)) #Hacemos broadcast del resultado a 3 canales 
        res_final = broad_res * frame  #Multiplicamos por el frame original
        writer.write(res_final)
    
    else:
        break
    
cap.release()
writer.release()